# SparkBasicsTask
> 1. Run the first cell to start the Spark Server
> 2. Run the second cell to excecute the wordcount code
> 3. Go to the SparkUI Url. How many job and stages are there and why?
> _One job, two stages. Two stages are needed because of the shuffle operation in the reduceByKey transformation. The data with the same key should land on the same cluster and has to be transferred over the network._
> 4. Copy/paste te wordcount code in a new cell.
> 5. In the coppied cells you have to add lines of code after the reduceByKey to also save the counts of first letter of the word (this way we know how many times a word starting with "a" appears in the text. The wordCount and letterCount must be saved seperately. After completing run the cell with the new code.
> _Tip: Python key values are represented by Tuples. To change the (word, count) tuple to (letter,count) you kan use a map function with (wordTuple[0][0],wordTuple[1]). The first [0] represents the first element of the tuple (the key). The second [0] is the first letter of the key string)
> 6. Go to the SparkUI Url. How many jobs are created? Why? Inspect the DAG of the 2 last jobs.
> _The engine creates a job for every "action" in the code. For every action it will create a new excecution plan and all tranformations will be processed twice._
> 7. Go to the Storage tab. Notice the tab is empty.
> 8. To avoid Spark from excecuting all the steps twice add wordCounts.cache() after the reduceByKey.
> 9. Run the code again. How many jobs are created now? Why? Inspect the DAG of the 2 last jobs. What is the difference with the previous DAGS? Compare the jobs by opening the DAGs in two different windows.
> 10. Go to the Storage tab. Notice the tab is not empty anymore. Inspect the storage tab.
> _By enabling caching, the first job will store the intermediate result. The second job will reuse this result, which will prevent processing the same steps twice. With caching, you can choose between processing and memory. When enough memory is available you can greatly increase the performance of the jobs._
> 11. Excecute sc.stop() to stop the Spark Server


In [6]:
import datetime
#Make sure pyspark is installed as a package of your project.
from pyspark import SparkConf, SparkContext
import ConnectionConfigKaloyan as kaloyanCC
import ConnectionConfig as ivanCC
ivanCC.setupEnvironment()
conf = SparkConf().setAppName("firstJob").setMaster("local[*]").setIfMissing("spark.logLineage", "true")
sc =SparkContext.getOrCreate((conf))
sc.uiWebUrl

'http://10.140.33.207:4041'

In [8]:
import datetime

lines = sc.textFile(".//FileStore//tables//shakespeare.txt")
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda word: len(word) > 0)
wordKv = words.map(lambda word: (word, 1))
wordCounts = wordKv.reduceByKey(lambda a,b:a +b)
wordCounts.saveAsTextFile("./output/words" + datetime.datetime.now().strftime("%m%d%Y%H%M%S"))

In [3]:
import datetime

lines = sc.textFile("./FileStore/tables/shakespeare.txt")
words = lines.flatMap(lambda line: line.split(" ")).filter(lambda word: len(word) > 0)
wordKv = words.map(lambda word: (word, 1))
wordCounts = wordKv.reduceByKey(lambda a,b:a +b)
#print(wordCounts.first())
wordCounts.cache()
letterCounts = wordCounts.map(lambda wordTuple: (wordTuple[0][0],wordTuple[1]) )
letterCounts = letterCounts.reduceByKey(lambda a,b:a +b)
wordCounts.saveAsTextFile("./output/words" + datetime.datetime.now().strftime("%m%d%Y%H%M%S"))
letterCounts.saveAsTextFile("./output/letters" + datetime.datetime.now().strftime("%m%d%Y%H%M%S"))


PythonRDD[26] at RDD at PythonRDD.scala:53

In [9]:
sc.stop()